# How to generate function arguments with the Chat Completions API

This notebook covers how to use the Chat Completions API to generate function arguments

`functions` is an optional parameter in the Chat Completion API which can be used to provide function specifications. The purpose of this is to enable models to generate function arguments which adhere to function input schemas. Note that the API will not actually execute any function calls. It is up to developers to execute function calls using model outputs.

If the `functions` parameter is provided then by default the model will decide when it is appropriate to use one of the functions. The API can be forced to use a specific function by setting the `function_call` parameter to `{"name": "<insert-function-name>"}`. The API can also be forced to not use any function by setting the `function_call` parameter to `"none"`. If a function is used, the output will contain `"finish_reason": "function_message"` in the response, as well as a `function_call` object that has the name of the function and the generated function arguments.

Functions are specified with the following fields:

- **Name:** The name of the function.
- **Description:** A description of what the function does. This will help the model decide when to use it.
- **Parameters:** The parameters object contains all of the input fields the function requires. These inputs can be of the following types: String, Number, Boolean, Object, Null, AnyOf. Refer to the [API reference docs](https://platform.openai.com/docs/api-reference/chat) for details.
- **Required:** Which of the parameters are required to make a query. The rest will be treated as optional.

In [1]:
!pip install openai
!pip install requests
!pip install termcolor
!pip install tenacity

In [2]:
import json
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo-0613"
EMBEDDING_MODEL = "text-embedding-ada-002"

## Utilities

First let's define a few utilities for making calls to the Chat Completions API and for maintaining and keeping track of the conversation state.

In [3]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [4]:
class Conversation:
    def __init__(self):
        self.conversation_history = []

    def add_message(self, role, content):
        message = {"role": role, "content": content}
        self.conversation_history.append(message)

    def display_conversation(self, detailed=False):
        role_to_color = {
            "system": "red",
            "user": "green",
            "assistant": "blue",
            "function": "magenta",
        }
        for message in self.conversation_history:
            print(
                colored(
                    f"{message['role']}: {message['content']}\n\n",
                    role_to_color[message["role"]],
                )
            )

## Basic concepts

Next we'll create a specification for a function called ```get_current_weather```. Later we'll pass this function specification to the API in order to generate function arguments that adhere to the specification.

In [5]:
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
            },
            "required": ["location", "format"],
        },
    },
    {
        "name": "get_n_day_weather_forecast",
        "description": "Get an N-day weather forecast",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "format": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                    "description": "The temperature unit to use. Infer this from the users location.",
                },
                "num_days": {
                    "type": "integer",
                    "description": "The number of days to forecast",
                }
            },
            "required": ["location", "format", "num_days"]
        },
    },
]


If we prompt the model about the current weather, it will respond with some clarifying questions.

In [6]:
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "What's the weather like today")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': 'Sure, could you please provide me with the location?'}

Once we provide the missing information, it will generate the appropriate function arguments for us.

In [7]:
conversation.add_message("user", "I'm in Glasgow, Scotland.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius"\n}'}}

By prompting it differently, we can get it to target the other function we've told it about.

In [8]:
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "what is the weather going to be like in Glasgow, Scotland over the next x days")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
assistant_message = chat_response.json()["choices"][0]["message"]
conversation.add_message(assistant_message["role"], assistant_message["content"])
assistant_message

{'role': 'assistant',
 'content': 'Sure, I can help you with that. Please provide me with the number of days you would like to forecast.'}

Once again, the model is asking us for clarification because it doesn't have enough information yet. In this case it already knows the location for the forecast, but it needs to know how many days are required in the forecast.

In [9]:
conversation.add_message("user", "5 days")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions
)
chat_response.json()["choices"][0]

{'index': 0,
 'message': {'role': 'assistant',
  'content': None,
  'function_call': {'name': 'get_n_day_weather_forecast',
   'arguments': '{\n  "location": "Glasgow, Scotland",\n  "format": "celsius",\n  "num_days": 5\n}'}},
 'finish_reason': 'function_call'}

We can force the model to use a specific function.

In [10]:
# by forcing the model to use a specific function, it is forced to make assumptions about how to use it
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me a weather report for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, function_call={"name": "get_n_day_weather_forecast"}
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_n_day_weather_forecast',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius",\n  "num_days": 1\n}'}}

In [11]:
# if we don't force the model to use get_n_day_weather_forecast it may not
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me a weather report for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, 
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': None,
 'function_call': {'name': 'get_current_weather',
  'arguments': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}}

We can also force the model to not use a function at all.

In [12]:
# if we force the model to not use a function, we prevent it from outputting a proper function call
conversation = Conversation()
conversation.add_message("system", "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.")
conversation.add_message("user", "Give me the current weather (use Celcius) for Toronto, Canada.")
chat_response = chat_completion_request(
    conversation.conversation_history, functions=functions, function_call="none"
)
chat_response.json()["choices"][0]["message"]


{'role': 'assistant',
 'content': '{\n  "location": "Toronto, Canada",\n  "format": "celsius"\n}'}